In [1]:
import pandas as pd
import numpy as np
import jieba

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from collections import Counter

In [4]:
df_t=pd.read_csv(r'C:\Users\a0970\OneDrive\Documents\datas\train.csv')

In [7]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320552 entries, 0 to 320551
Data columns (total 8 columns):
id           320552 non-null int64
tid1         320552 non-null int64
tid2         320552 non-null int64
title1_zh    320552 non-null object
title2_zh    320545 non-null object
title1_en    320552 non-null object
title2_en    320552 non-null object
label        320552 non-null object
dtypes: int64(3), object(5)
memory usage: 19.6+ MB


# Word Tokenizer

In [38]:
#use what we want column
col=['title1_zh','title2_zh','label']
train=df_t.loc[:,col]

In [45]:
#made a function
def cut_word(text):
    words=jieba.cut(text)
    return ' '.join(words)
    

In [48]:
#use apply to every sentence
# we need to use astype(str) to deal with some decode error
train['title1_token']=train.loc[:,'title1_zh'].astype(str).apply(cut_word)
train['title2_token']=train.loc[:,'title2_zh'].astype(str).apply(cut_word)

In [49]:
train.head()

,title1_zh,title2_zh,label,title1_token,title2_token
0,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,unrelated,2017 养老保险 又 新增 两项 ， 农村 老人 人人 可 申领 ， 你 领到 了 吗,警方 辟谣 “ 鸟巢 大会 每人 领 5 万 ” 仍 有 老人 坚持 进京
1,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,unrelated,""" 你 不来 深圳 ， 早晚 你 儿子 也 要 来 "" ， 不出 10 年 深圳 人均 GD...",深圳 GDP 首超 香港 ？ 深圳 统计局 辟谣 ： 只是 差距 在 缩小
2,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,unrelated,""" 你 不来 深圳 ， 早晚 你 儿子 也 要 来 "" ， 不出 10 年 深圳 人均 GD...",GDP 首超 香港 ？ 深圳 澄清 ： 还 差 一点点 … …
3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,unrelated,""" 你 不来 深圳 ， 早晚 你 儿子 也 要 来 "" ， 不出 10 年 深圳 人均 GD...",去年 深圳 GDP 首超 香港 ？ 深圳 统计局 辟谣 ： 还 差 611 亿
4,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,agreed,""" 用 大蒜 鉴别 地沟油 的 方法 , 怎么 鉴别 地沟油",吃 了 30 年 食用油 才 知道 ， 一片 大蒜 轻松 鉴别 地沟油


Use keras to assist us for the data preprocessing

In [50]:
import keras

C:\Users\a0970\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [51]:
#corpus=語料庫
corpus_x1=train.title1_token
corpus_x2=train.title2_token
corpus=pd.concat([corpus_x1,corpus_x2])


In [60]:
#show
pd.DataFrame(corpus.iloc[:5],columns=['title'])

,title
0,2017 养老保险 又 新增 两项 ， 农村 老人 人人 可 申领 ， 你 领到 了 吗
1,""" 你 不来 深圳 ， 早晚 你 儿子 也 要 来 "" ， 不出 10 年 深圳 人均 GD..."
2,""" 你 不来 深圳 ， 早晚 你 儿子 也 要 来 "" ， 不出 10 年 深圳 人均 GD..."
3,""" 你 不来 深圳 ， 早晚 你 儿子 也 要 来 "" ， 不出 10 年 深圳 人均 GD..."
4,""" 用 大蒜 鉴别 地沟油 的 方法 , 怎么 鉴别 地沟油"


In [62]:
#dictionary max word
MAX_limit_words= 10000
tokenizer=keras.preprocessing.text.Tokenizer(num_words=MAX_limit_words)

In [63]:
#build a dictionary
tokenizer.fit_on_texts(corpus)

In [65]:
#translate words into number by using dict we build
x1_train=tokenizer.texts_to_sequences(corpus_x1)
x2_train=tokenizer.texts_to_sequences(corpus_x2)

In [66]:
len(x1_train)

320552

In [67]:
x1_train[:1]

[[217, 1243, 37, 1150, 6003, 1, 30, 489, 2825, 126, 5576, 1, 10, 1803, 5, 19]]

In [69]:
#對應回來
for seq in x1_train[:1]:
    print([tokenizer.index_word[idx] for idx in seq])

['2017', '养老保险', '又', '新增', '两项', '，', '农村', '老人', '人人', '可', '申领', '，', '你', '领到', '了', '吗']


目前的所有序列有長有短，但為了讓NLP模型好跑，我們要讓所有的序列長度一致，Keras有相關的函數可以利用**也許這裡是可以改良的地方**

In [71]:
MAX_SEQUENCE_LENGTH=20
x1_train=keras.preprocessing.sequence.pad_sequences(x1_train,maxlen=MAX_SEQUENCE_LENGTH)
x2_train=keras.preprocessing.sequence.pad_sequences(x2_train,maxlen=MAX_SEQUENCE_LENGTH)

In [73]:
x1_train[0]

array([   0,    0,    0,    0,  217, 1243,   37, 1150, 6003,    1,   30,
        489, 2825,  126, 5576,    1,   10, 1803,    5,   19])

One-Hot Encoding 

In [74]:
label_to_index={
    'unrelated':0,
    'agreed':1,
    'disagreed':2
}

In [75]:
y_train=train.label.apply(lambda x :label_to_index[x])
y_train=np.asarray(y_train).astype('float32')

In [76]:
y_train[:5]

array([0., 0., 0., 0., 1.], dtype=float32)

In [80]:
#use keras to do one-hot encoding
y_train=keras.utils.to_categorical(y_train)

In [81]:
#pandas can use pd.dummies to do one-hot encoding
pd.get_dummies(train.label).head()

,agreed,disagreed,unrelated
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,1,0,0


In [82]:
y_train[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

切驗證資料集

In [83]:
from sklearn.model_selection import train_test_split

In [84]:
#validation_ratio 是要將資料切分成的比例
Validation_ratio=0.1
RANDOM_STATE=9527
#\換行繼續寫
x1_train,x1_val,x2_train,x2_val,y_train,y_val=\
    train_test_split(x1_train,x2_train,y_train,test_size=\
                     Validation_ratio,
                    random_state=RANDOM_STATE)

In [86]:
print("Training Set")
print("-" * 10)
print(f"x1_train: {x1_train.shape}")
print(f"x2_train: {x2_train.shape}")
print(f"y_train : {y_train.shape}")

print("-" * 10)
print(f"x1_val:   {x1_val.shape}")
print(f"x2_val:   {x2_val.shape}")
print(f"y_val :   {y_val.shape}")
print("-" * 10)
print("Test Set")

Training Set
----------
x1_train: (288496, 20)
x2_train: (288496, 20)
y_train : (288496, 3)
----------
x1_val:   (32056, 20)
x2_val:   (32056, 20)
y_val :   (32056, 3)
----------
Test Set


至此，資料前處理結束

# Use RNN

https://ithelp.ithome.com.tw/articles/10193469

NLP的重點是前後文會互相影響，因此RNN是特別設計來使用的，當輸入序列越長，向右展開的 RNN 也就越長。（模型也就需要訓練更久時間，這也是為何我們在資料前處理時設定了序列的最長長度）

但我們實際上使用的可能是LSTM(長短期記憶模型)

我們可以隨機初始化所有詞向量（如前述的隨機轉換），並利用平常訓練神經網路的反向傳播算法（Backpropagation），讓神經網路自動學到一組適合當前 NLP 任務的詞向量（如上張圖的理想狀態）。


詞嵌入（Word Embedding）:將一個詞彙或句子轉換成一個實數詞向量（Vectors of real numbers）的技術

In [91]:
# 基本參數設置，有幾個分類
NUM_CLASSES = 3

# 在語料庫裡有多少詞彙
MAX_NUM_WORDS = 10000

# 一個標題最長有幾個詞彙
MAX_SEQUENCE_LENGTH = 20

# 一個詞向量的維度
NUM_EMBEDDING_DIM = 256

# LSTM 輸出的向量維度
NUM_LSTM_UNITS = 128

孿生神經網路（Siamese Network）架構：

In [92]:
# 建立孿生 LSTM 架構（Siamese LSTM）
from keras import Input
from keras.layers import Embedding,LSTM,concatenate,Dense
from keras.models import Model
# 分別定義 2 個新聞標題 A & B 為模型輸入
# 兩個標題都是一個長度為 20 的數字序列
top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ), 
    dtype='int32')
bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ), 
    dtype='int32')
# 詞嵌入層
# 經過詞嵌入層的轉換，兩個新聞標題都變成
# 一個詞向量的序列，而每個詞向量的維度
# 為 256
embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)
top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)

# LSTM 層
# 兩個新聞標題經過此層後
# 為一個 128 維度向量
shared_lstm = LSTM(NUM_LSTM_UNITS)
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)

# 串接層將兩個新聞標題的結果串接單一向量
# 方便跟全連結層相連
merged = concatenate(
    [top_output, bm_output], 
    axis=-1)
# 全連接層搭配 Softmax Activation
# 可以回傳 3 個成對標題
# 屬於各類別的可能機率
dense =  Dense(
    units=NUM_CLASSES, 
    activation='softmax')
predictions = dense(merged)

# 我們的模型就是將數字序列的輸入，轉換
# 成 3 個分類的機率的所有步驟 / 層的總和
model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)



In [93]:
from keras.utils import plot_model
plot_model(
    model, 
    to_file='model.png', 
    show_shapes=True, 
    show_layer_names=False, 
    rankdir='LR')

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.

In [94]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 256)      2560000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 128)          197120      embedding_1[0][0]                
          

Softmax 函式一般都會被用在整個神經網路的最後一層上面，比方說我們這次的全連接層。

Softmax 函式能將某層中的所有神經元裡頭的數字作正規化（Normalization）：將它們全部壓縮到 0 到 1 之間的範圍，並讓它們的和等於 1。

為了讓機器自動「學習」，我們得給它一個損失函數（Loss Function），損失模型也是可以調整的地方

In [95]:
#利用交叉焍categorical_crossentropy 
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [97]:
# 決定一次要放多少成對標題給模型訓練
BATCH_SIZE = 512

# 決定模型要看整個訓練資料集幾遍
NUM_EPOCHS = 10

# 實際訓練模型
history = model.fit(
    # 輸入是兩個長度為 20 的數字序列
    x=[x1_train, x2_train], 
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    # 每個 epoch 完後計算驗證資料集
    # 上的 Loss 以及準確度
    validation_data=(
        [x1_val, x2_val], 
        y_val
    ),
    # 每個 epoch 隨機調整訓練資料集
    # 裡頭的數據以讓訓練過程更穩定
    shuffle=True
)

Train on 288496 samples, validate on 32056 samples
Epoch 1/10
288496/288496 [==============================] - 259s 899us/step - loss: 0.3640 - acc: 0.8372 - val_loss: 0.3799 - val_acc: 0.8304
Epoch 2/10
288496/288496 [==============================] - 257s 891us/step - loss: 0.3359 - acc: 0.8526 - val_loss: 0.3718 - val_acc: 0.8349
Epoch 3/10
288496/288496 [==============================] - 255s 885us/step - loss: 0.3134 - acc: 0.8647 - val_loss: 0.3679 - val_acc: 0.8369
Epoch 4/10
288496/288496 [==============================] - 257s 891us/step - loss: 0.2938 - acc: 0.8754 - val_loss: 0.3681 - val_acc: 0.8419
Epoch 5/10
288496/288496 [==============================] - 257s 889us/step - loss: 0.2768 - acc: 0.8847 - val_loss: 0.3584 - val_acc: 0.8459
Epoch 6/10
288496/288496 [==============================] - 258s 893us/step - loss: 0.2609 - acc: 0.8926 - val_loss: 0.3703 - val_acc: 0.8396
Epoch 7/10
288496/288496 [==============================] - 262s 908us/step - loss: 0.2463 - acc: